In [0]:
!pip install -U -q PyDrive
!pip install -q tf-nightly-2.0-preview

In [0]:
import os
import time
import pickle
import random 
import itertools
import warnings
import sys

import torch 
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import tensorflow as tf

from skimage.io import imread#,imshow
from skimage.filters import threshold_otsu
from skimage.morphology import convex_hull_image
from skimage.exposure import equalize_adapthist
from skimage.transform import resize
from skimage.color import grey2rgb

from matplotlib import pyplot as plt

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
%%capture
Dir = os.listdir("./")

if ("MC_Finger_Vein_Database" not in Dir):
    if ("Finger Vein Database.rar" not in Dir):
        checkpoint = drive.CreateFile({'id': '1Qq2TbwoolE9BGOD8Ooqon8KK1c_AkDhs'})
        checkpoint.GetContentFile('Finger_Vein_Database.zip')
#         !wget http://mla.sdu.edu.cn/sjk/Finger%20Vein%20Database.rar
        del checkpoint
    !unzip "Finger_Vein_Database.zip"
    !mv "Finger" "MC_Finger_Vein_Database"
del Dir

In [0]:
DATADIR = "MC_Finger_Vein_Database"
batch_size =  6*3*4

In [0]:
def load_image(addr):    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        image = grey2rgb(imread(addr))
#         image = image[:,5:-40]
#         thresh = threshold_otsu(image)
#         binary = image <= thresh
#         image = image * convex_hull_image(binary == 0)
#         image = equalize_adapthist(image)
        
#         del thresh, binary
#         return resize(image, (224,224,3), mode='constant' )/255.0
        return image/255.0

In [0]:
Dir = os.listdir("./")
# print(Dir)
if ("image_data.pickle" not in Dir and "test_image_data.pickle" not in Dir):
#     image_path = {"left_index":[], "left_ring":[], "left_middle":[],
#                  "right_index":[], "right_ring":[], "right_middle":[]}
#     test_image_path =  {"left_index":[], "left_ring":[], "left_middle":[],
#                         "right_index":[], "right_ring":[], "right_middle":[]}
    image_path = np.array([])
    test_image_path=np.array([])
    fingers={"MC_index":'1',"MC_middle":'2',"MC_ring":'3'}
    hands = {"left":'1',"right":'2'}
    n=len(os.listdir(DATADIR))
    remain=0
    for person in os.listdir(DATADIR):
        sys.stdout.write('\r Getting Data at-- {0} ---Remaining-- {1} '.format(str(person) ,str(n-remain)))
        sys.stdout.flush()
        remain+=1
        path=os.path.join(DATADIR,person)
        for hand in os.listdir(path):
            path_=os.path.join(path,hand)
            listofpath = os.listdir(path_)
            for finger in sorted(listofpath):
                label=int(person + hands[hand] + fingers[finger[:-6]])#001leftindex == 123
                if(int(n-remain)<21):
                    test_image_path = np.append(test_image_path, np.array([np.array([load_image(os.path.join(path_,finger)),label])]))
                else:
                    image_path = np.append(image_path, np.array([np.array([load_image(os.path.join(path_,finger)),label])]))

    del person, path, hand, finger, label
    image_path = image_path.reshape((3060, 2))
    test_image_path = test_image_path.reshape((756,2))
    import pickle
    
#     pickle_out = open("image_data.pickle","wb")
#     pickle.dump(image_path, pickle_out)
#     pickle_out.close()


#     pickle_out = open("test_image_data.pickle","wb")
#     pickle.dump(test_image_path, pickle_out)
#     pickle_out.close()
    
#     uploaded = drive.CreateFile({'title' : 'mc_image_data.pickle', "parents": 
#                                        [{"kind": "drive#childList","id": "11el4YYRbi7xpoNRbvbgE0LDo47ygBazP"}]})
#     uploaded.SetContentFile('image_data.pickle')
#     uploaded.Upload()
    
#     uploaded = drive.CreateFile({'title' : 'mc_test_image_path.pickle', "parents": 
#                                        [{"kind": "drive#childList","id": "11el4YYRbi7xpoNRbvbgE0LDo47ygBazP"}]})
#     uploaded.SetContentFile('test_image_path.pickle')
#     uploaded.Upload()
    
else:
    pickle_in = open("image_data.pickle","rb")
    image_path = pickle.load(pickle_in)
    
    pickle_in = open("test_image_data.pickle","rb")
    test_image_path = pickle.load(pickle_in)

del Dir

 Getting Data at-- 042 ---Remaining-- 1 

In [0]:
image_path = image_path.reshape((510,6,2))
np.random.shuffle(image_path)
image_path = image_path.reshape((3060,2))

In [0]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=64, dim=(32,32,32), #n_channels=1,
                 n_classes=10, shuffle=False):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
#         self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))#, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Image
            X[i,] = image_path[ID][0]

            # Label
            y[i] =  image_path[ID][1]

        return X, y

In [0]:
image_path[0][0].shape

(240, 320, 3)

In [0]:
dataset = DataGenerator(list(range(0, len(image_path))), batch_size=batch_size, dim = image_path[0][0].shape)

In [0]:
def _pairwise_distances(embeddings, squared=False):
    """Compute the 2D matrix of distances between all the embeddings.
    Args:
        embeddings: tensor of shape (batch_size, embed_dim)
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.
    Returns:
        pairwise_distances: tensor of shape (batch_size, batch_size)
    """
    # Get the dot product between all embeddings
    # shape (batch_size, batch_size)
    dot_product = tf.matmul(embeddings, tf.transpose(embeddings))

    # Get squared L2 norm for each embedding. We can just take the diagonal of `dot_product`.
    # This also provides more numerical stability (the diagonal of the result will be exactly 0).
    # shape (batch_size,)
    square_norm = tf.diag_part(dot_product)

    # Compute the pairwise distance matrix as we have:
    # ||a - b||^2 = ||a||^2  - 2 <a, b> + ||b||^2
    # shape (batch_size, batch_size)
    distances = tf.expand_dims(square_norm, 1) - 2.0 * dot_product + tf.expand_dims(square_norm, 0)

    # Because of computation errors, some distances might be negative so we put everything >= 0.0
    distances = tf.maximum(distances, 0.0)

    if not squared:
        # Because the gradient of sqrt is infinite when distances == 0.0 (ex: on the diagonal)
        # we need to add a small epsilon where distances == 0.0
        mask = tf.to_float(tf.equal(distances, 0.0))
        distances = distances + mask * 1e-16

        distances = tf.sqrt(distances)

        # Correct the epsilon added: set the distances on the mask to be exactly 0.0
        distances = distances * (1.0 - mask)

    return distances


def _get_anchor_positive_triplet_mask(labels):
    """Return a 2D mask where mask[a, p] is True iff a and p are distinct and have same label.
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    Returns:
        mask: tf.bool `Tensor` with shape [batch_size, batch_size]
    """
    # Check that i and j are distinct
    indices_equal = tf.cast(tf.eye(tf.shape(labels)[0]), tf.bool)
    indices_not_equal = tf.logical_not(indices_equal)

    # Check if labels[i] == labels[j]
    # Uses broadcasting where the 1st argument has shape (1, batch_size) and the 2nd (batch_size, 1)
    labels_equal = tf.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))

    # Combine the two masks
    mask = tf.logical_and(indices_not_equal, labels_equal)

    return mask


def _get_anchor_negative_triplet_mask(labels):
    """Return a 2D mask where mask[a, n] is True iff a and n have distinct labels.
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    Returns:
        mask: tf.bool `Tensor` with shape [batch_size, batch_size]
    """
    # Check if labels[i] != labels[k]
    # Uses broadcasting where the 1st argument has shape (1, batch_size) and the 2nd (batch_size, 1)
    labels_equal = tf.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))

    mask = tf.logical_not(labels_equal)

    return mask


def _get_triplet_mask(labels):
    """Return a 3D mask where mask[a, p, n] is True iff the triplet (a, p, n) is valid.
    A triplet (i, j, k) is valid if:
        - i, j, k are distinct
        - labels[i] == labels[j] and labels[i] != labels[k]
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    """
    # Check that i, j and k are distinct
    indices_equal = tf.cast(tf.eye(tf.shape(labels)[0]), tf.bool)
    indices_not_equal = tf.logical_not(indices_equal)
    i_not_equal_j = tf.expand_dims(indices_not_equal, 2)
    i_not_equal_k = tf.expand_dims(indices_not_equal, 1)
    j_not_equal_k = tf.expand_dims(indices_not_equal, 0)

    distinct_indices = tf.logical_and(tf.logical_and(i_not_equal_j, i_not_equal_k), j_not_equal_k)


    # Check if labels[i] == labels[j] and labels[i] != labels[k]
    label_equal = tf.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))
    i_equal_j = tf.expand_dims(label_equal, 2)
    i_equal_k = tf.expand_dims(label_equal, 1)

    valid_labels = tf.logical_and(i_equal_j, tf.logical_not(i_equal_k))

    # Combine the two masks
    mask = tf.logical_and(distinct_indices, valid_labels)

    return mask


def batch_all_triplet_loss(labels, embeddings, margin, squared=False):
    """Build the triplet loss over a batch of embeddings.
    We generate all the valid triplets and average the loss over the positive ones.
    Args:
        labels: labels of the batch, of size (batch_size,)
        embeddings: tensor of shape (batch_size, embed_dim)
        margin: margin for triplet loss
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.
    Returns:
        triplet_loss: scalar tensor containing the triplet loss
    """
    # Get the pairwise distance matrix
    pairwise_dist = _pairwise_distances(embeddings, squared=squared)

    # shape (batch_size, batch_size, 1)
    anchor_positive_dist = tf.expand_dims(pairwise_dist, 2)
    assert anchor_positive_dist.shape[2] == 1, "{}".format(anchor_positive_dist.shape)
    # shape (batch_size, 1, batch_size)
    anchor_negative_dist = tf.expand_dims(pairwise_dist, 1)
    assert anchor_negative_dist.shape[1] == 1, "{}".format(anchor_negative_dist.shape)

    # Compute a 3D tensor of size (batch_size, batch_size, batch_size)
    # triplet_loss[i, j, k] will contain the triplet loss of anchor=i, positive=j, negative=k
    # Uses broadcasting where the 1st argument has shape (batch_size, batch_size, 1)
    # and the 2nd (batch_size, 1, batch_size)
    triplet_loss = anchor_positive_dist - anchor_negative_dist + margin

    # Put to zero the invalid triplets
    # (where label(a) != label(p) or label(n) == label(a) or a == p)
    mask = _get_triplet_mask(labels)
    mask = tf.to_float(mask)
    triplet_loss = tf.multiply(mask, triplet_loss)

    # Remove negative losses (i.e. the easy triplets)
    triplet_loss = tf.maximum(triplet_loss, 0.0)

    # Count number of positive triplets (where triplet_loss > 0)
    valid_triplets = tf.to_float(tf.greater(triplet_loss, 1e-16))
    num_positive_triplets = tf.reduce_sum(valid_triplets)
    num_valid_triplets = tf.reduce_sum(mask)
    fraction_positive_triplets = num_positive_triplets / (num_valid_triplets + 1e-16)

    # Get final mean triplet loss over the positive valid triplets
    triplet_loss = tf.reduce_sum(triplet_loss) / (num_positive_triplets + 1e-16)

    return triplet_loss, fraction_positive_triplets


def batch_hard_triplet_loss(labels, embeddings, margin, squared=False):
    """Build the triplet loss over a batch of embeddings.
    For each anchor, we get the hardest positive and hardest negative to form a triplet.
    Args:
        labels: labels of the batch, of size (batch_size,)
        embeddings: tensor of shape (batch_size, embed_dim)
        margin: margin for triplet loss
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.
    Returns:
        triplet_loss: scalar tensor containing the triplet loss
    """
    # Get the pairwise distance matrix
    pairwise_dist = _pairwise_distances(embeddings, squared=squared)

    # For each anchor, get the hardest positive
    # First, we need to get a mask for every valid positive (they should have same label)
    mask_anchor_positive = _get_anchor_positive_triplet_mask(labels)
    mask_anchor_positive = tf.to_float(mask_anchor_positive)

    # We put to 0 any element where (a, p) is not valid (valid if a != p and label(a) == label(p))
    anchor_positive_dist = tf.multiply(mask_anchor_positive, pairwise_dist)

    # shape (batch_size, 1)
    hardest_positive_dist = tf.reduce_max(anchor_positive_dist, axis=1, keepdims=True)
    tf.summary.scalar("hardest_positive_dist", tf.reduce_mean(hardest_positive_dist))

    # For each anchor, get the hardest negative
    # First, we need to get a mask for every valid negative (they should have different labels)
    mask_anchor_negative = _get_anchor_negative_triplet_mask(labels)
    mask_anchor_negative = tf.to_float(mask_anchor_negative)

    # We add the maximum value in each row to the invalid negatives (label(a) == label(n))
    max_anchor_negative_dist = tf.reduce_max(pairwise_dist, axis=1, keepdims=True)
    anchor_negative_dist = pairwise_dist + max_anchor_negative_dist * (1.0 - mask_anchor_negative)

    # shape (batch_size,)
    hardest_negative_dist = tf.reduce_min(anchor_negative_dist, axis=1, keepdims=True)
    tf.summary.scalar("hardest_negative_dist", tf.reduce_mean(hardest_negative_dist))

    # Combine biggest d(a, p) and smallest d(a, n) into final triplet loss
    triplet_loss = tf.maximum(hardest_positive_dist - hardest_negative_dist + margin, 0.0)

    # Get final mean triplet loss
    triplet_loss = tf.reduce_mean(triplet_loss)

    return triplet_loss

In [0]:
def create_embedding_network(input_shape):
    input_ = tf.keras.layers.Input(input_shape)
    x= tf.keras.layers.Conv2D(64, (3, 3), strides=(1, 1), padding = "valid", name="conv1", activation="relu" , input_shape= input_shape[1:])(input_)
    x= tf.keras.layers.MaxPooling2D(pool_size= (3, 3), strides = (1, 1), padding = "valid", name="pool1")(x)

#     x= tf.keras.layers.Conv2D(64, (1, 1), strides=(1, 1), padding = "valid", name="conv2a", activation="relu")(x)
#     x= tf.keras.layers.Conv2D(64, (3, 3), strides=(1, 1), padding = "valid", name="conv2", activation="relu")(x)
#     x= tf.keras.layers.MaxPooling2D(pool_size= (2, 2), strides = (2, 2), padding = "valid", name="pool2")(x)
    
    x= tf.keras.layers.Conv2D(64, (1, 1), strides=(1, 1), padding = "valid", name="conv2a", activation="relu")(x)
    x= tf.keras.layers.Conv2D(64, (3, 3), strides=(1, 1), padding = "valid", name="conv2", activation="relu")(x)
    x= tf.keras.layers.MaxPooling2D(pool_size= (3, 3), strides = (2, 2), padding = "valid", name="pool2")(x)

    x= tf.keras.layers.Conv2D(128, (1, 1), strides=(1, 1), padding = "valid", name="conv3a", activation="relu")(x)
    x= tf.keras.layers.Conv2D(128, (3, 3), strides=(1, 1), padding = "valid", name="conv3", activation="relu")(x)
    x= tf.keras.layers.MaxPooling2D(pool_size= (3, 3), strides = (2, 2), padding = "valid", name="pool3")(x)
    x= tf.keras.layers.Conv2D(256, (1, 1), strides=(1, 1), padding = "valid", name="conv4a", activation="relu")(x)
    x= tf.keras.layers.Conv2D(256, (3, 3), strides=(1, 1), padding = "valid", name="conv4", activation="relu")(x)
    
    x= tf.keras.layers.MaxPooling2D(pool_size= (3, 3), strides = (2, 2), padding = "valid", name="pool4")(x)
    x= tf.keras.layers.Conv2D(256, (1, 1), strides=(1, 1), padding = "valid", name="conva5a", activation="relu")(x)
    x= tf.keras.layers.Conv2D(256, (3, 3), strides=(1, 1), padding = "valid", name="conva5", activation="relu")(x)
    x= tf.keras.layers.MaxPooling2D(pool_size= (3, 3), strides = (2, 2), padding = "valid", name="pool5")(x)
    x= tf.keras.layers.Flatten()(x)
    x= tf.keras.layers.Dense(1024,activation=tf.nn.softmax)(x)
    model = tf.keras.models.Model(inputs=input_, outputs=x)
    return model

In [0]:
def build_model(input_shape):
    global learning_rate
    anchor = tf.keras.layers.Input(shape=input_shape)
    embedding_network = create_embedding_network(input_shape)
    loss = batch_hard_triplet_loss(positive_embedding)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    model.compile(loss=loss, optimizer=optimizer)
    return model


In [0]:
# def get_triplets(features):
#     df = pd.DataFrame(features)
#     triplets = []
#     for index, row in df.iterrows():
#     same_tag = df.loc[df.iloc[:, -1] == row.iloc[-1]]
#     same_tag_indexes = list(set(same_tag.index) - {index})
#     diff_tag_indexes = list(set(df.index) - set(same_tag_indexes) - {index})
#     anchor = row[0]
#     pos = df.iloc[random.choice(same_tag_indexes), :].iloc[0]
#     neg = df.iloc[random.choice(diff_tag_indexes), :].iloc[0]
#     triplets.append(list(list([anchor, pos, neg])))
#     return np.array(triplets)


In [0]:
model = build_model((240,320,1))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 240, 320, 1)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 238, 318, 64)      640       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 236, 316, 64)      0         
_________________________________________________________________
conv2a (Conv2D)              (None, 236, 316, 64)      4160      
_________________________________________________________________
conv2 (Conv2D)               (None, 234, 314, 64)      36928     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 116, 156, 64)      0         
_________________________________________________________________
conv3a (Conv2D)              (None, 116, 156, 128)     8320      
__________

TypeError: ignored

In [0]:
# df = pd.DataFrame(training_data)
# epochs = 1

# for i in range(num_epochs):
#     hard=[]
#     history = model.fit([data[:,0],data[:,1],data[:,2]], y=np.zeros(data[:,0].shape[0]), epochs=epochs, batch_size=1)
#     #   epochs+=1
#     epoch_loss = history.history['loss'][-1]
#     file_name = 'Epoch_'+str(i)+'.h5'
#     model.save_weights(file_name)
#     model_file = drive.CreateFile({'title' : file_name})
#     model_file.SetContentFile(file_name)
#     model_file.Upload()

#     for index, row in df.iterrows():
#         same_tag = df.loc[df.iloc[:, -1] == row.iloc[-1]]
#         same_tag_indexes = list(set(same_tag.index) - {index})
#         diff_tag_indexes = list(set(df.index) - set(same_tag_indexes) - {index})
#         anchor = row[0]
#         pos = df.iloc[random.choice(same_tag_indexes), :].iloc[0]
#         choice = random.choice(diff_tag_indexes)
#         neg = df.iloc[choice, :].iloc[0]
#         anchor_ = anchor.reshape((1,)+anchor.shape)
#         pos_ = pos.reshape((1,)+pos.shape)
#         neg_ = neg.reshape((1,)+neg.shape)
#         ans = model.predict([anchor_,pos_,neg_])
#     if ans[0] > epoch_loss:
#         hard.append(list(list([anchor, pos, neg])))
#     if(len(hard)<1):
#         break
#     data = np.array(hard)
    

8/8 [==============================] - 0s 31ms/sample - loss: 0.2340


In [0]:

# st = time.time()
# model.predict([data[:5,0],data[1900:1905,1],data[90:95,2]])
# print(time.time()-st)



array([0., 0., 0., 0., 0.], dtype=float32)

In [0]:
# ans = model.predict([data[:7,0],data[0:7,1],data[:7,2]])